In [10]:
import sys
import joblib

sys.path.append('../')

from Modules.Utils.Imports import *
from Modules.Utils.ModelWrapper import ModelWrapper
from Modules.Models.BuildBINNs import BINNCovasim

import Modules.Loaders.DataFormatter as DF
import datetime

from utils import plot_loss_convergence, get_case_name
import matplotlib
matplotlib.use('Agg')

device = torch.device(GetLowestGPU(pick_from=[0,1,2,3]))

Device set to cpu


In [11]:
path = '../Data/covasim_data/drums_data/'
population = 50000
test_prob = 0.1
trace_prob = 0.3
keep_d = True
retrain = False
dynamic=True
chi_type = 'piecewise' # constant, piecewise, linear, sin
n_runs = 1000
case_name = get_case_name(population, test_prob, trace_prob, keep_d, dynamic=dynamic, chi_type=chi_type)

params = DF.load_covasim_data(path, population, test_prob, trace_prob, keep_d, case_name + '_' + str(n_runs), plot=True)

In [12]:
def to_torch(ndarray):
    arr = torch.tensor(ndarray, dtype=torch.float)
    arr.requires_grad_(True)
    arr = arr.to(device)
    return arr

def to_numpy(x):
    return x.detach().cpu().numpy()

In [13]:
data = np.array(params['data'])
data_smooth = np.mean(data, axis=0)
data_smooth = (data_smooth / params['population'])

#N = len(data_smooth) - 2
N = len(data_smooth)
#u = to_torch(data_smooth[1:N+1,:])
u = data_smooth

t_max_real = N
t = np.arange(N)[:,None] + 1

params.pop('data')
tracing_array = params['tracing_array']

In [14]:
# split into train/val and convert to torch
split = int(0.8*N)
# generate shuffled array of indices from 0 to N-1
p = np.random.permutation(N)

x_train = to_torch(p[:split][:, None]/(N-1))
y_train = to_torch(u[p[:split]])
x_val = to_torch(p[split:][:, None]/(N-1))
y_val = to_torch(u[p[split:]])

In [15]:
# generate save path
mydir = os.path.join('../models/covasim/nondenoised', datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
os.makedirs(mydir)

# initialize model
binn = BINNCovasim(params, N, tracing_array, keep_d=keep_d, chi_type=chi_type)
binn.to(device)

BINNCovasim(
  (surface_fitter): main_MLP(
    (mlp): BuildMLP(
      (activation): ReLU()
      (output_activation): ReLU()
      (MLP): Sequential(
        (0): Linear(in_features=1, out_features=512, bias=True)
        (1): ReLU()
        (2): Dropout(p=0.2, inplace=False)
        (3): Linear(in_features=512, out_features=256, bias=True)
        (4): ReLU()
        (5): Dropout(p=0.2, inplace=False)
        (6): Linear(in_features=256, out_features=256, bias=True)
        (7): ReLU()
        (8): Dropout(p=0.2, inplace=False)
        (9): Linear(in_features=256, out_features=9, bias=True)
        (10): ReLU()
      )
    )
    (softmax): Softmax(dim=1)
  )
  (eta_func): infect_rate_MLP(
    (mlp): BuildMLP(
      (activation): ReLU()
      (output_activation): Sigmoid()
      (MLP): Sequential(
        (0): Linear(in_features=3, out_features=256, bias=True)
        (1): ReLU()
        (2): Dropout(p=0.2, inplace=False)
        (3): Linear(in_features=256, out_features=1, bias=True)


In [16]:
parameters = binn.parameters()
opt = torch.optim.Adam(parameters, lr=1e-3)
os.makedirs(os.path.join(mydir, case_name))
model = ModelWrapper(
    model=binn,
    optimizer=opt,
    loss=binn.loss,
    augmentation=None,
    # scheduler= scheduler,
    save_name=os.path.join(mydir, case_name) )
model.str_name = 'STEAYDQRF'

In [17]:
# save the range information before training
ranges = [binn.yita_lb, binn.yita_ub, binn.beta_lb, binn.beta_ub, binn.tau_lb, binn.tau_ub]
file_name = '_'.join([str(m) for m in ranges])
joblib.dump(None, os.path.join(mydir, file_name)) # model.save_folder
# if retrain
if retrain:
    model.load(model.save_name + '_best_val_model', device=device)
    model.model.train()
    model.save_name += '_retrain'
    
epochs = int(2e3)
batch_size = 128
rel_save_thresh = 0.05

In [18]:
# train jointly
model.fit(
    x=x_train,
    y=y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=None,
    verbose=1,
    validation_data=[x_val, y_val],
    early_stopping=20000,
    rel_save_thresh=rel_save_thresh)

Epoch 672 | Train loss = 1.6300e+00 | Val loss = 1.1221e+00 | Remaining = 2:24:17           

KeyboardInterrupt: 

In [ ]:
total_train_losses = model.train_loss_list
total_val_losses = model.val_loss_list

plot_loss_convergence(total_train_losses, total_val_losses, rel_save_thresh, model.save_folder)